In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
taxi = sc.textFile('Data/yellow_tripdata_2016-06.csv', use_unicode=False).cache()

In [5]:
import csv

In [22]:
weather = sc.textFile('Data/WeatherData2016Clean.csv', use_unicode=False).cache()

In [23]:
list(enumerate(weather.first().split(',')))

[(0, ''), (1, 'Date'), (2, 'DateVal'), (3, 'Temp'), (4, 'Conditions')]

In [8]:
list(enumerate(taxi.first().split(',')))

[(0, 'VendorID'),
 (1, 'tpep_pickup_datetime'),
 (2, 'tpep_dropoff_datetime'),
 (3, 'passenger_count'),
 (4, 'trip_distance'),
 (5, 'pickup_longitude'),
 (6, 'pickup_latitude'),
 (7, 'RatecodeID'),
 (8, 'store_and_fwd_flag'),
 (9, 'dropoff_longitude'),
 (10, 'dropoff_latitude'),
 (11, 'payment_type'),
 (12, 'fare_amount'),
 (13, 'extra'),
 (14, 'mta_tax'),
 (15, 'tip_amount'),
 (16, 'tolls_amount'),
 (17, 'improvement_surcharge'),
 (18, 'total_amount')]

In [28]:
import datetime
def weather_hours_mapper(id, data):
    if id == 0:
        data.next()
    for row in csv.reader(data):
        yield row
        
weatherRDD = weather.mapPartitionsWithIndex(weather_hours_mapper)

In [29]:
weatherRDD.take(3)

[['0', '01/01/2016', '1', '38', 'Normal'],
 ['1', '01/02/2016', '2', '36', 'Normal'],
 ['2', '01/03/2016', '3', '40', 'Normal']]

In [26]:
weather.take(3)

[',Date,DateVal,Temp,Conditions',
 '0,01/01/2016,1,38,Normal',
 '1,01/02/2016,2,36,Normal']

In [12]:
list(enumerate(taxi.first().split(',')))
import calendar

In [13]:
# Clean Taxi Data
def parserTaxi(id, data):
    if id == 0:
        data.next()
    for i in csv.reader(data):
        tpep_pickup_datetime, tpep_dropoff_datetime, pickup_longitude, \
        pickup_latitude, dropoff_longitude, dropoff_latitude, Date, Weekday = i[1], i[2], i[5], i[6], i[9], i[10], \
        datetime.datetime.strptime(i[1], "%Y-%m-%d %H:%M:%S").date(), \
        calendar.day_name[datetime.datetime.strptime("2016-06-09 21:06:36", "%Y-%m-%d %H:%M:%S").weekday()]
        # rows required for the taxi data
        yield tpep_pickup_datetime, tpep_dropoff_datetime, pickup_longitude, \
        pickup_latitude, dropoff_longitude, dropoff_latitude, Date, Weekday


In [14]:
taxiRDD = taxi.mapPartitionsWithIndex(parserTaxi)

In [15]:
taxiRDD.take(5)

[('2016-06-09 21:06:36',
  '2016-06-09 21:13:08',
  '-73.983360290527344',
  '40.760936737060547',
  '-73.977462768554688',
  '40.753978729248047',
  datetime.date(2016, 6, 9),
  'Thursday'),
 ('2016-06-09 21:06:36',
  '2016-06-09 21:35:11',
  '-73.981719970703125',
  '40.736667633056641',
  '-73.981636047363281',
  '40.670242309570313',
  datetime.date(2016, 6, 9),
  'Thursday'),
 ('2016-06-09 21:06:36',
  '2016-06-09 21:13:10',
  '-73.994316101074219',
  '40.751071929931641',
  '-74.004234313964844',
  '40.742168426513672',
  datetime.date(2016, 6, 9),
  'Thursday'),
 ('2016-06-09 21:06:36',
  '2016-06-09 21:36:10',
  '-73.98236083984375',
  '40.773891448974609',
  '-73.929466247558594',
  '40.851539611816406',
  datetime.date(2016, 6, 9),
  'Thursday'),
 ('2016-06-09 21:06:36',
  '2016-06-09 21:23:23',
  '-73.987106323242187',
  '40.733173370361328',
  '-73.985908508300781',
  '40.766445159912109',
  datetime.date(2016, 6, 9),
  'Thursday')]

In [16]:
import datetime
import calendar
calendar.day_name[datetime.datetime.strptime("2016-06-09 21:06:36", "%Y-%m-%d %H:%M:%S").weekday()]

'Thursday'

In [17]:
taxiDF = taxiRDD.toDF()

In [18]:
taxiDF.head()

Row(_1=u'2016-06-09 21:06:36', _2=u'2016-06-09 21:13:08', _3=u'-73.983360290527344', _4=u'40.760936737060547', _5=u'-73.977462768554688', _6=u'40.753978729248047', _7=datetime.date(2016, 6, 9), _8=u'Thursday')

In [20]:
# removing all the 0 lat data
taxiDF = taxiDF.filter(taxiDF._3 == 0)

In [30]:
weatherDF = weatherRDD.toDF()

In [31]:
weatherDF.head()

Row(_1=u'0', _2=u'01/01/2016', _3=u'1', _4=u'38', _5=u'Normal')